SERIALIZERS.PY

In [ ]:
class Profile(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE, null=True)
    bio = models.TextField(blank=True)  # Changed to TextField for longer descriptions
    image = models.ImageField(upload_to='profileimgs', blank=True, null=True, default='profileimgs/ifiok.png')
    joined = models.DateTimeField(auto_now_add=True)
    followers = models.ManyToManyField(User, related_name='followers', blank=True)
    following = models.ManyToManyField(User, related_name='followers', blank=True)

SERIALIZERS.PY

In [ ]:
from .models import *

class ProfileSerializer(ModelSerializer):
    class Meta:
        model = Profile
        fields = '__all__'

VIEWS.PY

In [ ]:
@api_view(['GET', 'POST'])
@permission_classes([IsAuthenticated])
def userprofiles(request):
    if request.method == 'GET':
        query = request.GET.get('query', '')
        # e.g 127.0.0.1:8000/userprofiles/?query=john
        profiles = Profile.objects.filter(Q(username__icontains=query))
        serializer = ProfileSerializer(profiles, many=True)
        return Response(serializer.data, status=status.HTTP_200_OK)

    if request.method == 'POST':
        serializer = ProfileSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    
@api_view(['GET', 'PUT', 'DELETE', 'PATCH'])
@permission_classes([IsAuthenticated])
def userprofiledetail(request, pk):
    try:
        profile = Profile.objects.get(pk=pk)
    except Profile.DoesNotExist:
        return Response({"detail": "Profile not found"}, status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        serializer = ProfileSerializer(profile, many=False)
        return Response(serializer.data, status=status.HTTP_200_OK)

    if request.method == 'PUT':
        serializer = ProfileSerializer(profile, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_200_OK)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
    if request.method == 'PUT':
        serializer = ProfileSerializer(profile, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_200_OK)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
    if request.method == 'PATCH':
        serializer = ProfileSerializer(profile, data=request.data, partial=True)
        serializer = ProfileSerializer(profile, data=request.data, partial=True)

    if request.method == 'DELETE':
        profile.delete()
        return Response({"detail": "Profile deleted successfully"}, status=status.HTTP_204_NO_CONTENT)

URLS.PY

In [ ]:
path('userprofiles/', views.userprofiles, name= "userprofiles"),
path('userprofile/<int:pk>', views.userprofiledetail, name='userprofiledetail'),